In [1]:
from PIL import Image
from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
import pandas as pd
import os
from shutil import copyfile
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

Using TensorFlow backend.


In [42]:
# path to the model weights files.
weights_path = '/media/tyler/slowdata/models/keras_vgg_16'
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = '/home/tyler/data/kaggle/dog_breed/train/'
ordered_train_folder = "/home/tyler/data/kaggle/dog_breed/train_ordered/"
test_data_dir = '/home/tyler/data/kaggle/dog_breed/test/'
labels = '/home/tyler/data/kaggle/dog_breed/labels.csv'

epochs = 1000
batch_size = 64
nb_train_samples = 10222

# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False,
                          input_shape=(img_width, img_height, 3), classes=120)
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(120, activation='softmax'))

# add the model on top of the convolutional base
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:19]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

Model loaded.


In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [21]:
labels_df = pd.read_csv(labels)
labels_df = labels_df.set_index('id')
class_names = labels_df['breed'].unique()
n_classes = len(class_names)
index_to_classes = {i:c for i, c in enumerate(class_names)}
classes_to_index = {c:i for i,c in index_to_classes.items()}
print("{} classes".format(n_classes))

120 classes


In [36]:
all_ys = []
all_Xs = []
for img_path in os.listdir(train_data_dir):
    class_name = img_path.split(".")[0]
    y = classes_to_index[labels_df.loc[class_name]['breed']]
    img = image.load_img(train_data_dir + img_path, 
                         target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    all_ys.append(y)
    all_Xs.append(x)

In [37]:
X = np.array(all_Xs)
X = X.reshape(-1, img_width, img_height, 3)

In [38]:
y = np.array(all_ys)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow(X, y, batch_size=batch_size)

# fine-tune the model
checkpointer = ModelCheckpoint(filepath='./tmp/weights.{epoch:02d}.hdf5', verbose=1,
                              save_best_only=True)

model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    callbacks=[checkpointer],
    validation_data = (X[:100], y[:100]))

/home/tyler/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=1000, callbacks=[<keras.ca..., validation_data=(array([[[..., steps_per_epoch=159)`


Epoch 1/1000
159/159 [==============================] - 53s - loss: 4.7839 - acc: 0.0103 - val_loss: 15.7421 - val_acc: 0.0100
Epoch 2/1000
159/159 [==============================] - 52s - loss: 4.7797 - acc: 0.0112 - val_loss: 15.7406 - val_acc: 0.0000e+00
Epoch 3/1000
159/159 [==============================] - 51s - loss: 4.7803 - acc: 0.0117 - val_loss: 15.7202 - val_acc: 0.0000e+00
Epoch 4/1000
159/159 [==============================] - 51s - loss: 4.7748 - acc: 0.0122 - val_loss: 15.7074 - val_acc: 0.0000e+00
Epoch 5/1000
159/159 [==============================] - 51s - loss: 4.7697 - acc: 0.0140 - val_loss: 15.6829 - val_acc: 0.0000e+00
Epoch 6/1000
159/159 [==============================] - 51s - loss: 4.7706 - acc: 0.0116 - val_loss: 15.6568 - val_acc: 0.0100
Epoch 7/1000
159/159 [==============================] - 51s - loss: 4.7655 - acc: 0.0136 - val_loss: 15.6579 - val_acc: 0.0200
Epoch 8/1000
159/159 [==============================] - 51s - loss: 4.7596 - acc: 0.0140 - val_

159/159 [==============================] - 52s - loss: 4.6241 - acc: 0.0324 - val_loss: 15.2511 - val_acc: 0.0200
Epoch 29/1000
159/159 [==============================] - 52s - loss: 4.6229 - acc: 0.0350 - val_loss: 15.1659 - val_acc: 0.0200
Epoch 30/1000
159/159 [==============================] - 52s - loss: 4.6095 - acc: 0.0358 - val_loss: 15.1544 - val_acc: 0.0200
Epoch 31/1000
159/159 [==============================] - 51s - loss: 4.6066 - acc: 0.0343 - val_loss: 15.1026 - val_acc: 0.0200
Epoch 32/1000
159/159 [==============================] - 51s - loss: 4.5907 - acc: 0.0389 - val_loss: 15.0560 - val_acc: 0.0200
Epoch 33/1000
159/159 [==============================] - 52s - loss: 4.5829 - acc: 0.0411 - val_loss: 14.9664 - val_acc: 0.0200
Epoch 34/1000
159/159 [==============================] - 52s - loss: 4.5675 - acc: 0.0365 - val_loss: 14.9726 - val_acc: 0.0300
Epoch 35/1000
159/159 [==============================] - 52s - loss: 4.5724 - acc: 0.0431 - val_loss: 14.9531 - val_ac

159/159 [==============================] - 52s - loss: 4.3571 - acc: 0.0626 - val_loss: 14.4043 - val_acc: 0.0400
Epoch 58/1000
159/159 [==============================] - 52s - loss: 4.3207 - acc: 0.0742 - val_loss: 14.3624 - val_acc: 0.0400
Epoch 59/1000
159/159 [==============================] - 52s - loss: 4.3229 - acc: 0.0734 - val_loss: 14.3399 - val_acc: 0.0400
Epoch 60/1000
159/159 [==============================] - 52s - loss: 4.3138 - acc: 0.0709 - val_loss: 14.3313 - val_acc: 0.0400
Epoch 61/1000
159/159 [==============================] - 52s - loss: 4.3042 - acc: 0.0705 - val_loss: 14.1451 - val_acc: 0.0400
Epoch 62/1000
159/159 [==============================] - 52s - loss: 4.2926 - acc: 0.0771 - val_loss: 14.1694 - val_acc: 0.0400
Epoch 63/1000
159/159 [==============================] - 52s - loss: 4.2858 - acc: 0.0774 - val_loss: 14.1593 - val_acc: 0.0400
Epoch 64/1000
159/159 [==============================] - 52s - loss: 4.2736 - acc: 0.0809 - val_loss: 14.1405 - val_ac

159/159 [==============================] - 52s - loss: 4.0355 - acc: 0.1096 - val_loss: 13.5642 - val_acc: 0.0800
Epoch 87/1000
159/159 [==============================] - 52s - loss: 4.0419 - acc: 0.1040 - val_loss: 13.5401 - val_acc: 0.0900
Epoch 88/1000
159/159 [==============================] - 52s - loss: 4.0326 - acc: 0.1084 - val_loss: 13.5762 - val_acc: 0.0900
Epoch 89/1000
159/159 [==============================] - 52s - loss: 4.0139 - acc: 0.1125 - val_loss: 13.5269 - val_acc: 0.0900
Epoch 90/1000
159/159 [==============================] - 52s - loss: 4.0142 - acc: 0.1140 - val_loss: 13.4445 - val_acc: 0.0900
Epoch 91/1000
159/159 [==============================] - 52s - loss: 4.0016 - acc: 0.1116 - val_loss: 13.4083 - val_acc: 0.0900
Epoch 92/1000
159/159 [==============================] - 52s - loss: 3.9861 - acc: 0.1136 - val_loss: 13.3049 - val_acc: 0.0900
Epoch 93/1000
159/159 [==============================] - 51s - loss: 3.9708 - acc: 0.1179 - val_loss: 13.3051 - val_ac

159/159 [==============================] - 52s - loss: 3.7895 - acc: 0.1421 - val_loss: 12.2153 - val_acc: 0.1300
Epoch 115/1000
159/159 [==============================] - 52s - loss: 3.7886 - acc: 0.1468 - val_loss: 12.2911 - val_acc: 0.1200
Epoch 116/1000
159/159 [==============================] - 52s - loss: 3.7612 - acc: 0.1494 - val_loss: 12.2500 - val_acc: 0.1200
Epoch 117/1000
159/159 [==============================] - 52s - loss: 3.7712 - acc: 0.1471 - val_loss: 12.2570 - val_acc: 0.1100
Epoch 118/1000
159/159 [==============================] - 52s - loss: 3.7597 - acc: 0.1496 - val_loss: 12.2240 - val_acc: 0.1200
Epoch 119/1000
159/159 [==============================] - 52s - loss: 3.7346 - acc: 0.1504 - val_loss: 12.2284 - val_acc: 0.1300
Epoch 120/1000
159/159 [==============================] - 52s - loss: 3.7434 - acc: 0.1515 - val_loss: 12.1153 - val_acc: 0.1300
Epoch 121/1000
159/159 [==============================] - 52s - loss: 3.7225 - acc: 0.1523 - val_loss: 12.1396 -

In [ ]:
if not train_model:
    model = load_model('./tmp/weights.49.hdf5')

In [16]:
class_to_index = train_generator.class_indices
index_to_class = {i:c for c,i in class_to_index.items()}

In [19]:
def get_predictions(img_path):
    img = image.load_img(img_path, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return model.predict(x)

In [22]:
rows = []
for img_path in os.listdir(test_data_dir):
    predictions = list(get_predictions(test_data_dir + img_path))
    predictions.insert(0, img_path.split(".")[0])
    rows.append(predictions)

In [30]:
clean_rows = []
for r in rows:
    predictions = list(r[1])
    predictions.insert(0, r[0])
    clean_rows.append(predictions)

In [38]:
df = pd.DataFrame(clean_rows, columns=['id'] + list(class_to_index.keys()))

In [41]:
df.to_csv("test_results.csv", index=False)